<a href="https://colab.research.google.com/github/WittmannF/machine-learning-web-app/blob/master/3-machine-learning/3_machine_learning_production_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## Imports
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LassoLars, HuberRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.ensemble import StackingRegressor

In [0]:
## Parameters
URL_TREINO = 'https://raw.githubusercontent.com/WittmannF/machine-learning-web-app/master/2-preprocessamento/treino_preprocessado.csv'
URL_TESTE = 'https://raw.githubusercontent.com/WittmannF/machine-learning-web-app/master/2-preprocessamento/teste_preprocessado.csv'


In [0]:
## Functions
def read_train_test():
    treino = pd.read_csv(URL_TREINO)
    teste = pd.read_csv(URL_TESTE)


    X_train = treino.drop('preco', axis=1)
    y_train = treino['preco']
    X_test = teste.drop('preco', axis=1)
    y_test = teste['preco']
    return X_train, y_train, X_test, y_test

In [0]:
## Execution
X_train, y_train, X_test, y_test = read_train_test()

In [0]:
param = {'kernel' : ('linear', 'poly', 'rbf'),
         'C' : [1,5,10]},

svr = SVR()

gs_svr = GridSearchCV(svr, 
                     param, 
                     cv=5)

gs_svr.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=({'C': [1, 5, 10],
                          'kernel': ('linear', 'poly', 'rbf')},),
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [0]:
gs_svr.score(X_test, y_test)

0.7460944176955905

In [0]:
lgb_params = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40, 100],
    'num_leaves': [6, 16, 31],
    }

param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}

lgb = LGBMRegressor()

gs_lgb = GridSearchCV(lgb, lgb_params, cv=5)
gs_lgb.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.01, 0.1, 1],
                         'n_estimators': [20, 40, 100],
                         'num_leaves': [6, 16, 31]},
             

In [0]:
gs_lgb.score(X_test, y_test)

0.7419033049737831

In [0]:
regs = [gs_svr, gs_lgb] #SVR, LGB

all_preds = []
for reg in regs:
    all_preds.append(reg.predict(X_test))

In [0]:
y_pred = np.mean(all_preds, axis=0)
r2_score(y_test, y_pred)

0.7471582687352167

In [0]:
estimators = [
    ('svr', gs_svr),
    ('lgb', gs_lgb)
]
reg = StackingRegressor(
    estimators=estimators
)

reg.fit(X_train, y_train)

StackingRegressor(cv=None,
                  estimators=[('svr',
                               GridSearchCV(cv=5, error_score=nan,
                                            estimator=SVR(C=1.0, cache_size=200,
                                                          coef0=0.0, degree=3,
                                                          epsilon=0.1,
                                                          gamma='scale',
                                                          kernel='rbf',
                                                          max_iter=-1,
                                                          shrinking=True,
                                                          tol=0.001,
                                                          verbose=False),
                                            iid='deprecated', n_jobs=None,
                                            param_grid=({'C': [1, 5, 10],
                                                       

In [0]:
reg.score(X_test, y_test)

0.7473729524826268

In [0]:
# Exportar modelo
pickle.dump(reg, open('regressor.pkl', 'wb'), protocol=4)

In [0]:
# Vamos testar se o classificador está funcionando mesmo
# Primeiro resetar todas as variaveis
%reset -f

In [0]:
import pickle

In [0]:
reg = pickle.load(open('regressor.pkl', 'rb'))

In [0]:
reg

StackingRegressor(cv=None,
                  estimators=[('svr',
                               GridSearchCV(cv=5, error_score=nan,
                                            estimator=SVR(C=1.0, cache_size=200,
                                                          coef0=0.0, degree=3,
                                                          epsilon=0.1,
                                                          gamma='scale',
                                                          kernel='rbf',
                                                          max_iter=-1,
                                                          shrinking=True,
                                                          tol=0.001,
                                                          verbose=False),
                                            iid='deprecated', n_jobs=None,
                                            param_grid=({'C': [1, 5, 10],
                                                       

In [0]:
import numpy as np
np.expm1(reg.predict([[0, 0, 1, np.log1p(2), np.log1p(120)]]))

array([2703.32326175])

In [0]:
np.expm1(1.098612)

1.9999991339957957

In [0]:
np.expm1(4.795791)

120.00005498280686